In [36]:
import gc
import logging
import os
import glob
import pandas as pd

import sys
# sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')


import time

from collections import defaultdict

import torch
import torch.nn as nn
import torch.optim as optim
from math import exp

import numpy as np
torch.backends.cudnn.benchmark = True

from matplotlib import pyplot as plt
import matplotlib as mpl
import matplotlib.patches as patches
from matplotlib import pyplot as plt
from argoverse.map_representation.map_api import ArgoverseMap
from argoverse.data_loading.argoverse_forecasting_loader import ArgoverseForecastingLoader
from argoverse.visualization.visualize_sequences import viz_sequence
avm = ArgoverseMap()

num = 10

KeyboardInterrupt: 

In [2]:
import os
import sys
sys.path.append("../ddn/")
sys.path.append("./")
import warnings
warnings.filterwarnings('ignore')

import torch
import numpy as np
import scipy.special
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

from scipy.linalg import block_diag
from torch.utils.data import Dataset, DataLoader
#from bernstein import bernstesin_coeff_order10_new

from argoverse.map_representation.map_api import ArgoverseMap
from argoverse.data_loading.argoverse_forecasting_loader import ArgoverseForecastingLoader
from argoverse.visualization.visualize_sequences import viz_sequence
avm = ArgoverseMap()

def denoise(gt_x, gt_y, w = 7):
    # denoising
    gt_x_t = []
    gt_y_t = []
    for iq in range(len(gt_x)):
        if iq >= w and iq + w <= len(gt_x):
            gt_x_t.append(np.mean(gt_x[iq: iq + w]))
            gt_y_t.append(np.mean(gt_y[iq: iq + w]))
        elif iq < w:
            okx = np.mean(gt_x[w: w + w])
            gt_x_t.append(gt_x[0] + (okx - gt_x[0]) * (iq) / w)
            oky = np.mean(gt_y[w: w + w])
            gt_y_t.append(gt_y[0] + (oky - gt_y[0]) * (iq) / w)
        else:
            okx = np.mean(gt_x[len(gt_x) - w:len(gt_x) - w  + w])
            oky = np.mean(gt_y[len(gt_x) - w: len(gt_x) - w + w])
            gt_x_t.append(okx + (gt_x[-1] - okx) * (w - (len(gt_x) - iq)) / w)
            gt_y_t.append(oky + (gt_y[-1] - oky) * (w - (len(gt_y) - iq)) / w)                   

    gt_x = gt_x_t
    gt_y = gt_y_t
    return gt_x, gt_y

def rotate(gt_x, gt_y,theta):
    gt_x_x = [ (gt_x[k] * np.cos(theta) - gt_y[k] * np.sin(theta))  for k in range(len(gt_x))]
    gt_y_y = [ (gt_x[k] * np.sin(theta) + gt_y[k] * np.cos(theta))  for k in range(len(gt_x))]
    gt_x = gt_x_x
    gt_y = gt_y_y
    return gt_x, gt_y

def transform(x_traj, y_traj, dt = 0.3, t_obs = 20, theta=None, offsets=None):
    if offsets == None:
        x_traj -= x_traj[0]
        y_traj -= y_traj[0]
    else:
        x_traj -= offsets[0]
        y_traj -= offsets[1]
        
    gt_x = x_traj
    gt_y = y_traj
    
    gt_x, gt_y = denoise(gt_x, gt_y)
    v_x = [ (gt_x[k + 1] - gt_x[k])/dt  for k in range(len(gt_x) - 1)]
    v_y = [ (gt_y[k + 1] - gt_y[k])/dt  for k in range(len(gt_y) - 1)]
    psi = [ np.arctan2(v_y[k], v_x[k]) for k in range(len(v_x))]  

    # till here, gt-> (50, 1), v -> (49, 1), psi -> (31, 1)

    # obtain this -psi
    if theta == None:
        theta = -psi[t_obs - 1]

    # rotate by theta
    gt_x, gt_y = rotate(gt_x, gt_y, theta)
    v_x = [ (gt_x[k + 1] - gt_x[k])/dt  for k in range(len(gt_x) - 1)]
    v_y = [ (gt_y[k + 1] - gt_y[k])/dt  for k in range(len(gt_y) - 1)]
    psi = [ np.arctan2(v_y[k], v_x[k]) for k in range(len(v_x))]
    psidot = [ (psi[k + 1] - psi[k])/dt for k in range(len(psi) - 1) ]
    psi_traj = [i.item() for i in psi]
    psidot_traj = [i.item() for i in psidot]
    
    return gt_x, gt_y, v_x, v_y, psi, psidot, psi_traj, psidot_traj, theta

In [3]:
random_colors = np.random.rand(100,3)
left_lane_changes = []
right_lane_changes = []
left_forks = []
right_forks = []
holds = []

In [34]:
import random
import networkx as nx
from shapely.geometry.polygon import Polygon, Point
data_path="/home/vikrant/datasets/datasets/argoverse/val/data"
output_dir="../results/"
t_obs=20
dt=0.3
t_obs=20
pred=False
pred_array=None
batch_size = 512
dpi=100
w,h=512,512
res=0.5
paths = glob.glob(os.path.join(data_path, "*.csv"))
print(paths[0])
trajs = []
color = {
    'polygon': '#e6cf93',
    'polygon-outline': '#e6cf93',
    'centerline': '#fceec7',
    'agent': 'blue',
    'av': 'grey',
    'other': 'grey',
    'outline': 'black'
}
color = {
    'polygon': 'white',
    'polygon-outline': 'white',
    'centerline': 'white',
    'agent': 'white',
    'av': 'white',
    'other': 'white',
    'outline': 'black'
}

numss = []

data_path="/home/vikrant/datasets/datasets/argoverse/val/data"
order = np.load('../data/order.npy')

from tqdm import tqdm
for idx in tqdm(range(len(paths))):
    if idx < 16742: continue
    fil = order[idx].split('/')[-1]
    path = data_path + "/" + fil
    dff = pd.read_csv(path)
    
    city = dff['CITY_NAME'].values[0]    
    
    agent_df = dff[dff['OBJECT_TYPE'] == 'AGENT']
    x_a = agent_df['X'].values
    y_a = agent_df['Y'].values    
    x_a, y_a = denoise(x_a, y_a)
    
    theta_init = np.arctan2(y_a[20] - y_a[19], x_a[20] - x_a[19])
    x_rotated, y_rotated = rotate(x_a - x_a[0], y_a - y_a[0], -theta_init)
    
    # group by timestamp
    dfs = [x for _, x in dff.groupby('TIMESTAMP')]    
    
    total_successors = []
    current = []
    das_polygons = []
    das_polygons_mp = []
    das_ids = []

    agent_polygons = []    
    dic = {}
    G = nx.DiGraph()
    cur = None
    
    color_map = []
#     print(color_mapping)


    orderr = {}
    traj_path = {}
    curr_order = 0    


    #########################################################################    
    ##################### BUILDING GRAPH ####################################
    #########################################################################
    prev_lane_id = None
    fg = False
    for indd in range(0, 50):
        lane_id = avm.get_nearest_centerline(np.array([x_a[indd],y_a[indd]]), city_name=city)[0].id
        if indd > 0:
            lneigh, rneigh = avm.get_lane_segment_adjacent_ids(lane_segment_id=prev_lane_id, city_name=city)[0], avm.get_lane_segment_adjacent_ids(lane_segment_id=prev_lane_id, city_name=city)[1]
            if lneigh == lane_id:
                left_lane_changes.append(idx)
                fg = True
                break
            elif rneigh == lane_id:
                right_lane_changes.append(idx)
                fg = True
                break
        prev_lane_id = lane_id
    if fg is False:
        # not performed lane change
        angle = np.arctan2(y_rotated[49] - y_rotated[19],x_rotated[49] - x_rotated[19]) - np.arctan2(y_rotated[19] - y_rotated[18],x_rotated[19] - x_rotated[18])
        if angle > np.pi/3:
            # right fork
            right_forks.append(idx)
        elif angle < -np.pi/3:
            left_forks.append(idx)
        else:
            holds.append(idx)

  0%|          | 0/39472 [00:00<?, ?it/s]

/home/vikrant/datasets/datasets/argoverse/val/data/37076.csv


100%|██████████| 39472/39472 [1:45:14<00:00,  6.25it/s]  


In [33]:
# 16742
print(len(left_lane_changes))
print(len(holds))
print(len(right_lane_changes))
print(len(left_forks))
print(len(right_forks))

1128
15600
0
5
9


In [35]:
# 16742
print(len(left_lane_changes))
print(len(holds))
print(len(right_lane_changes))
print(len(left_forks))
print(len(right_forks))

2387
35327
1723
8
27


In [ ]:
    colors = ["grey", "blue", "orange", "green", "green", "green"]
    seq = []
    left_forks = []
    right_forks = []
    left_lane_change = []
    right_lane_change = []
    holds = []
    init_len = len(trajs)
    for leafno, leaf in enumerate(leaves):
        # go above
        cnttt = 0
        lane_id = leaf
        path = []
        plt.subplot(len(leaves)/2 + 2, 2, leafno + 3) # row 1, col 2 index 1
        for arr in avm.find_local_lane_polygons([x_min, x_max, y_min, y_max], city):
            x_plot, y_plot = arr[:, 0], arr[:, 1]
            x_plot, y_plot = rotate(x_plot - x_a[0], y_plot - y_a[0], -theta_init)
            plt.plot(x_plot, y_plot, color='black', zorder=4)        
        while paths.get(lane_id) != None and cnttt < 100:
            cnttt = cnttt + 1
            polygon = avm.get_lane_segment_polygon(lane_id, city)
            x_plot, y_plot = polygon[:, 0], polygon[:, 1]
            x_plot, y_plot = rotate(x_plot - x_a[0], y_plot - y_a[0], -theta_init)
            plt.fill(x_plot, y_plot, color=colors[cnttt-1], zorder=5)                    
            path.append(lane_id)
            if paths.get(lane_id) != None:
                lane_id = paths[lane_id]
        seq.append(path)
        if len(path) == 0: continue
        plt.xlim([x_traj[20] - 50, x_traj[20] + 50])
        plt.ylim([y_traj[20] - 50, y_traj[20] + 50])
        straight = []
        centerline = avm.get_lane_segment_centerline(path[0], city)
        x_plot, y_plot = centerline[:, 0], centerline[:, 1]
        x_plot, y_plot = rotate(x_plot - x_a[0], y_plot - y_a[0], -theta_init)
        cx, cy = x_plot[-1], y_plot[-1]
        angle = np.arctan2(cy - y_rotated[20], cx - x_rotated[20]) - np.arctan2(y_rotated[20] - y_rotated[19], x_rotated[20] - x_rotated[19])
        if cur in path:
            if angle > np.pi/3:
                left_forks.append(path)
                plt.title("Left fork: {}".format(len(left_forks)))
            elif angle < -np.pi/3:
                right_forks.append(path)
                plt.title("Right fork: {}".format(len(right_forks)))
            else:
                holds.append(path)
                plt.title("Straight")
        else:
            if angle > 0:
                left_lane_change.append(path)
                plt.title("Left lane change: {}".format(len(left_lane_change)))
            else:
                right_lane_change.append(path)
                plt.title("Right lane change: {}".format(len(right_lane_change)))
        plt.plot(x_plot, y_plot, color="red", zorder=11)
#         plt.scatter(x_plot[-1], y_plot[-1], color="red", zorder=11)
        plt.scatter(cx, cy, color="red", zorder=11)
        centerline = avm.get_lane_segment_centerline(path[1], city)
        x_plot, y_plot = centerline[:, 0], centerline[:, 1]
        x_plot, y_plot = rotate(x_plot - x_a[0], y_plot - y_a[0], -theta_init)
        cx, cy = x_plot[-1], y_plot[-1]
        psi_init = np.arctan2(y_rotated[20] - y_rotated[19], x_rotated[20] - x_rotated[19])
        psi_init_1 = np.arctan2(y_rotated[19] - y_rotated[18], x_rotated[19] - x_rotated[18])
        v_init = ((y_rotated[20] - y_rotated[19])/0.1)**0.5 + ((x_rotated[20] - x_rotated[19])/0.1)**0.5
        psidot_init = (psi_init_1 - psi_init)/0.1
        psi_fin = np.arctan2(y_plot[-1] - y_plot[-2], x_plot[-1] - x_plot[-2])
        var_inp = np.array([cx, cy, psi_fin])
        fixed_params = np.array([x_rotated[20], y_rotated[20], 0, psi_init, psidot_init])
        var_inp = np.array([cx, cy, psi_fin])

        sol, _ = problem.solve(fixed_params.reshape(1,5), var_inp.reshape(1,3))

        tot_time = np.linspace(0.0, t_fin, num)
        tot_time_copy = tot_time.reshape(num, 1)

        P, Pdot, Pddot = bernstein_coeff_order10_new(10, tot_time_copy[0], tot_time_copy[-1], tot_time_copy)

        nvar = 11
        x_pred = np.dot(P, sol[:, :nvar].T)
        y_pred = np.dot(P, sol[:, nvar:2*nvar].T)
        x_pred = x_pred.T
        y_pred = y_pred.T
        plt.scatter(x_pred, y_pred, color='yellow', zorder=12)
        
        plt.scatter(cx, cy, color="red", zorder=11)
        if abs(angle + np.pi) > 0.3:
            x_final = np.concatenate((x_rotated[:20], x_pred[0]))
            y_final = np.concatenate((y_rotated[:20], y_pred[0]))
#             trajs.append(np.concatenate((x_final.reshape(50, 1), y_final.reshape(50, 1)), axis=1))


In [71]:
np.save("./trajs.npy", trajs)
np.save("./nums.npy", numss)

In [76]:
np.load("trajs.npy").shape

(128, 50, 2)

In [75]:
np.load("/home/vikrant/datasets/datasets/argoverse/val_data.npy", allow_pickle=True).shape

(39472, 50, 2)